# Visualize trained ad hoc performances of the agents
The purpose of this notebook is to evaluate the performances of different agent setups, as explained in the paper.
For this, we first load all different agents and their according training weights, and let them play against each other in several different setups. The results are then furhter plotted inside heatmaps etc.(Please refer to the paper for a more detailed explanation)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from training.tf_agents_lib.pyhanabi_env_wrapper import PyhanabiEnvWrapper
from rainbow_adhoc_player import RainbowAdHocRLPlayer
from tf_agent_adhoc_player_ppo import PPOTfAgentAdHocPlayer
from tf_agent_adhoc_player_reinforce import ReinforceTfAgentAdHocPlayer
from simulate_adhoc_game import AdHocExperiment

In [ ]:
# weight locations
ppo_rootdir = "../agents/trained_models/ppo/"
reinforce_rootdir = "../agents/trained_models/reinforce/train/policy/"

# AdHoc experiment
num_players = 4
game_type = "Hanabi-Full"
observation_size = 1041
max_moves = 38

num_episodes = 100

rainbow_player_custom_r = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "custom_r1",layers=1)
rainbow_player_standard_1 = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "10k",layers=1)
rainbow_player_standard_2 = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "20k",layers=1)
rainbow_player_rule_teacher_1 = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "team1_adhoc",layers=2)
rainbow_player_rule_teacher_2 = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "team2_adhoc",layers=2)
rainbow_player_rule_teacher_3 = RainbowAdHocRLPlayer(observation_size, num_players, max_moves, "team3_adhoc",layers=1)

ppo_agent = PPOTfAgentAdHocPlayer(ppo_rootdir, game_type, num_players)
reinforce_agent = ReinforceTfAgentAdHocPlayer(reinforce_rootdir, game_type, num_players)

agents = [rainbow_player_standard_1, 
          rainbow_player_standard_2, 
          rainbow_player_custom_r,
          rainbow_player_rule_teacher_1,
          rainbow_player_rule_teacher_2,
          rainbow_player_rule_teacher_3,
          ppo_agent, 
          reinforce_agent]


agent_names = [
                 "Rainbow_standard_1", "Rainbow_standard_2", 
                 "Rainbow_standard_custom_R", "Rainbow_rule_teacher_1", 
                 "Rainbow_rule_teacher_2", "Rainbow_rule_teacher_3",
                 "PPO", "Reinforce"
              ]

In [ ]:
df_avg = pd.DataFrame(columns=["1_vs_3"]+agent_names)
df_max = pd.DataFrame(columns=["1_vs_3"]+agent_names)

In [ ]:
df_avg["1_vs_3"] = agent_names
df_max["1_vs_3"] = agent_names

In [ ]:
arr_prob_dists = np.zeros((len(agent_names),num_episodes))

In [ ]:
for i, agent_class in enumerate(agents):
    for j, test_agent in enumerate(agents):
        players = [agent_class for _ in range(num_players-1)] + [test_agent]
        tmp_game = AdHocExperiment(game_type, players)
        tmp_result_list, tmp_avg = tmp_game.run_experiment(num_episodes)
        df_avg.iloc[j][i+1] = tmp_avg
        df_max.iloc[j][i+1] = max(tmp_result_list)       
        if i == j:
            arr_prob_dists[i] = tmp_result_list

In [ ]:
max_val_array = df_max.values[:,1:].astype('float32')
avg_val_array = df_avg.values[:,1:].astype('float32')

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (N, M).
    row_labels
        A list or array of length N with the labels for the rows.
    col_labels
        A list or array of length M with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A list or array of two color specifications.  The first is used for
        values below a threshold, the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
agent_ids = ["Agent-1","Agent-2","Agent-3","Agent-4","Agent-5","Agent-6","Agent-7","Agent-8"]
team_ids = ["Team-1","Team-2","Team-3","Team-4","Team-5","Team-6","Team-7","Team-8"]

In [ ]:
fig, ax = plt.subplots()

im, cbar = heatmap(avg_val_array, agent_ids, team_ids, ax=ax, cbarlabel="average score")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

# Results are saved
fig.tight_layout()
plt.savefig("adhoc_avg_score.png")
plt.show()

In [ ]:
fig, ax = plt.subplots()

im, cbar = heatmap(max_val_array, agent_ids, team_ids, ax=ax, cbarlabel="max. score")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

# Results are saved
fig.tight_layout()
plt.savefig("adhoc_max_score.png")
plt.show()

In [ ]:
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 28}

matplotlib.rc('font', **font)

In [ ]:

fig, (ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8) = plt.subplots(1,8,figsize=(80,10))

ax1.hist(arr_prob_dists[0],density=True,bins=int(max(arr_prob_dists[0])))
ax1.set_title("Agent-1")
ax2.hist(arr_prob_dists[1],density=True,bins=int(max(arr_prob_dists[1])))
ax2.set_title("Agent-2")
ax3.hist(arr_prob_dists[2],density=True,bins=int(max(arr_prob_dists[2])))
ax3.set_title("Agent-3")
ax4.hist(arr_prob_dists[3],density=True,bins=int(max(arr_prob_dists[3])))
ax4.set_title("Agent-4")
ax5.hist(arr_prob_dists[4],density=True,bins=int(max(arr_prob_dists[4])))
ax5.set_title("Agent-5")
ax6.hist(arr_prob_dists[5],density=True,bins=int(max(arr_prob_dists[5])))
ax6.set_title("Agent-6")
ax7.hist(arr_prob_dists[6],density=True,bins=int(max(arr_prob_dists[6])))
ax7.set_title("Agent-7")
ax8.hist(arr_prob_dists[7],density=True,bins=int(max(arr_prob_dists[7])))
ax8.set_title("Agent-8")
plt.savefig("probabilities.png")